In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
from sklearn.metrics import roc_auc_score, auc

In [2]:
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [4]:
act_train_df = pd.read_csv('./red-hat_data/act_train.csv', dtype={'people_id': np.str, 'activity_id': np.str, 'outcome': np.int8}, parse_dates=['date'])
act_test_df = pd.read_csv('./red-hat_data/act_test.csv', dtype={'people_id': np.str, 'activity_id': np.str}, parse_dates=['date'])
ppl_df = pd.read_csv('./red-hat_data/people.csv', dtype={'people_id': np.str, 'activity_id': np.str, 'char_38': np.int32}, parse_dates=['date'])

In [5]:
test_id = act_test_df.activity_id

In [6]:
train_df = pd.read_csv('./processing_data/train.csv')
test_df = pd.read_csv('./processing_data/test.csv', )

In [7]:
X_train = train_df.drop(['outcome'], axis=1)
Y_train = train_df['outcome']

In [8]:
whole_df = pd.concat([X_train,test_df],ignore_index=True,sort=False)

In [9]:
objective_var=['group_1','activity_category','char_1_x','char_2_x','char_3_x','char_4_x','char_5_x','char_6_x','char_7_x','char_8_x','char_9_x','char_10_x','char_1_y','char_2_y','char_3_y','char_4_y','char_5_y','char_6_y','char_7_y','char_8_y','char_9_y']

In [10]:
def reduce_dim(df, col, toreplace):
    for index,i in df[col].duplicated(keep=False).iteritems():
        if i==False:
            df.set_value(index,col,toreplace)
    return df

In [11]:
for var in objective_var:
    X_whole = reduce_dim(whole_df, var, 9999999)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [12]:
X = X_whole[:len(train_df)]
X_test = X_whole[len(train_df):]

In [13]:
del act_train_df, act_test_df, ppl_df
del train_df, test_df
del whole_df, X_whole

In [15]:
le = LabelEncoder()
enc = OneHotEncoder(handle_unknown='ignore')

In [16]:
not_object = []
for var in X.columns:
    if var not in objective_var:
        not_object.append(var)
    else:
        temp = pd.concat([X[var],X_test[var]], sort=False)        
        le.fit(temp.values)
        X[var] = le.transform(X[var].values)
        X_test[var] = le.transform(X_test[var].values) 

In [17]:
enc = enc.fit(pd.concat([X[objective_var],X_test[objective_var]]))
X_obj_sparse = enc.transform(X[objective_var])
X_test_obj_sparse = enc.transform(X_test[objective_var])

In [18]:
X[not_object].shape, X[objective_var].shape, X_test[not_object].shape, X_test[objective_var].shape

((2197291, 38), (2197291, 21), (498687, 38), (498687, 21))

In [19]:
from scipy.sparse import hstack

X_sparse = hstack((X[not_object], X_obj_sparse))
X_test_sparse = hstack((X_test[not_object], X_test_obj_sparse))

In [20]:
X_sparse.shape, X_test_sparse.shape

((2197291, 37439), (498687, 37439))

In [21]:
dtrain = xgb.DMatrix(X_sparse,label=Y_train)
dtest = xgb.DMatrix(X_test_sparse)

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [22]:
param = {'max_depth':10, 'eta':0.02, 'silent':0, 'objective':'binary:logistic' }
#param['nthread'] = 2
param['eval_metric'] = 'auc'
param['subsample'] = 0.7
param['colsample_bytree']= 0.7
param['min_child_weight'] = 0
param['booster'] = "gblinear"

In [25]:
# np.random.seed(120)
evals  = [(dtrain,'train')]
num_round = 500
bst = xgb.train(param, dtrain, num_round, evals, early_stopping_rounds=10, verbose_eval=10)

[0]	train-auc:0.890781
Will train until train-auc hasn't improved in 10 rounds.
[10]	train-auc:0.96075
[20]	train-auc:0.987145
[30]	train-auc:0.992606
[40]	train-auc:0.994636
[50]	train-auc:0.995679
[60]	train-auc:0.996289
[70]	train-auc:0.996669
[80]	train-auc:0.996915
[90]	train-auc:0.997079
[100]	train-auc:0.997189
[110]	train-auc:0.997266
[120]	train-auc:0.997322
[130]	train-auc:0.997364
[140]	train-auc:0.997398
[150]	train-auc:0.997426
[160]	train-auc:0.997449
[170]	train-auc:0.997467
[180]	train-auc:0.997482
[190]	train-auc:0.997495
[200]	train-auc:0.997506
[210]	train-auc:0.997516
[220]	train-auc:0.997525
[230]	train-auc:0.997532
[240]	train-auc:0.997539
[250]	train-auc:0.997545
[260]	train-auc:0.99755
[270]	train-auc:0.997555
[280]	train-auc:0.99756
[290]	train-auc:0.997564
[300]	train-auc:0.997567
[310]	train-auc:0.99757
[320]	train-auc:0.997573
[330]	train-auc:0.997576
[340]	train-auc:0.997578
[350]	train-auc:0.99758
[360]	train-auc:0.997582
[370]	train-auc:0.997584
[380]	tra

In [30]:
y_pred_bst = bst.predict(dtest)

In [31]:
submission4 = pd.DataFrame({'activity_id' : test_id, 'outcome': y_pred_bst})
submission4.to_csv('submission4.csv', index=False)

In [32]:
# drop unimportance variables and try the aboove again

In [33]:
x_drop_rfe = ['activity_id','char_1_x','char_2_x','char_3_x','char_4_x','char_5_x','char_8_x','char_9_x','char_10_x','day_x','group_1','char_7_y','char_9_y','char_29','char_31','day_y']
x_drop_original = ['char_1_x','char_2_x','char_3_x','char_4_x','char_5_x','char_6_x','char_7_x','char_8_x','char_9_x','char_10_x','group_1']

In [34]:
objective_var1 = ['activity_category','char_1_y','char_2_y','char_3_y','char_4_y','char_5_y','char_6_y','char_7_y','char_8_y','char_9_y']

In [35]:
objective_var2 = ['group_1','activity_category','char_1_x','char_2_x','char_3_x','char_4_x','char_5_x','char_6_x','char_7_x','char_8_x','char_9_x','char_1_y','char_2_y','char_3_y','char_4_y','char_5_y','char_6_y','char_7_y','char_8_y','char_9_y']

In [109]:
X1 = X.drop('char_10_x', axis=1)
X1_test = X_test.drop('char_10_x', axis=1)

In [110]:
not_object1 = []
for var in X1.columns:
    if var not in objective_var2:
        not_object1.append(var)
    else:
        temp = pd.concat([X1[var],X1_test[var]], sort=False)
        le = LabelEncoder()
        le.fit(temp.values)
        X1[var] = le.transform(X1[var].values)
        X1_test[var] = le.transform(X1_test[var].values) 

In [111]:
enc = OneHotEncoder(handle_unknown='ignore')
enc = enc.fit(pd.concat([X1[objective_var2],X1_test[objective_var2]]))
X1_obj_sparse = enc.transform(X1[objective_var2])
X1_test_obj_sparse = enc.transform(X1_test[objective_var2])

In [112]:
X1[not_object1].shape, X1[objective_var2].shape, X1_test[not_object1].shape, X1_test[objective_var2].shape

((2197291, 38), (2197291, 20), (498687, 38), (498687, 20))

In [113]:
from scipy.sparse import hstack

X1_sparse = hstack((X1[not_object1], X1_obj_sparse))
X1_test_sparse = hstack((X1_test[not_object1], X1_test_obj_sparse))

In [114]:
X1_sparse.shape, X1_test_sparse.shape

((2197291, 31273), (498687, 31273))

In [115]:
dtrain1 = xgb.DMatrix(X1_sparse,label=Y_train)
dtest1 = xgb.DMatrix(X1_test_sparse)

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [120]:
param1 = {'max_depth':10, 'eta':0.02, 'silent':0, 'objective':'binary:logistic' }
param['nthread'] = 4
param1['eval_metric'] = 'auc'
param1['subsample'] = 0.6
param1['colsample_bytree']= 0.7
param1['min_child_weight'] = 2
param1['booster'] = "gbtree"

In [24]:
# np.random.seed(120)
evals1  = [(dtrain1,'train')]
num_round = 400
bst1 = xgb.train(param1, dtrain1, num_round, evals1, early_stopping_rounds=10, verbose_eval=10)

NameError: name 'dtrain1' is not defined

In [118]:
y_pred_bst1 = bst1.predict(dtest1)

In [119]:
submission5 = pd.DataFrame({'activity_id' : test_id, 'outcome': y_pred_bst1 })
submission5.to_csv('submission5.csv', index = False)

In [122]:
# RF

In [125]:
RF = RandomForestClassifier()
RF.fit(X_sparse, Y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [127]:
RF.score(X_sparse, Y_train)

0.9996318193630247

In [128]:
y_pred_sparse_RF = RF.predict(X_test_sparse)

In [129]:
submission7 = pd.DataFrame({'activity_id' : test_id, 'outcome': y_pred_sparse_RF})
submission7.to_csv('submission7.csv', index = False)

In [26]:
LSVC = LinearSVC()

In [ ]:
LSVC.fit(X_sparse, Y_train)

In [28]:
y_pred_sparse_LSVC = LSVC.predict(X_test_sparse)

In [29]:
submission8 = pd.DataFrame({'activity_id' : test_id, 'outcome': y_pred_sparse_LSVC})
submission8.to_csv('submission8.csv', index = False)

In [36]:
ftp = SelectFromModel(LSVC)

In [46]:
pactTrainFeatured = ftp.transform(X_sparse)

In [ ]:
features = ftp.get_support(indices = False)

In [41]:
c =[]
for i in range(len(features)): 
    if features[i] == False:
        c.append(i) 

In [45]:
# columNames = list(dtest.columns.values)
print(dtest.[c])

TypeError: 'DMatrix' object is not subscriptable